In [1]:
import pandas as pd
import json

In [3]:
## sorted sort -k2 -k4 -r -n <paired_concepts_test_file.txt >> pc_sorted_test.txt
paired_concepts_table = pd.read_csv('pc_test.csv')
paired_concepts_table

,dataset_id,concept_id_1,concept_id_2,concept_count,concept_prevalence,chi_square_t,chi_square_p,expected_count,ln_ratio,rel_freq_1,rel_freq_2
0,1,8507,955372,6,0.000003,0.698369,4.033323e-01,4.631562,0.258865,0.000008,0.545454
1,1,8507,985786,7,0.000004,0.414192,5.198494e-01,8.421023,-0.184821,0.000009,0.350000
2,1,8507,990029,7,0.000004,2.040442,1.531652e-01,10.526280,-0.407965,0.000009,0.280000
3,1,8507,924939,8,0.000004,0.036359,8.487762e-01,8.421023,-0.051290,0.000011,0.400000
4,1,8507,974677,8,0.000004,2.014034,1.558505e-01,5.473665,0.379493,0.000011,0.615385
...,...,...,...,...,...,...,...,...,...,...,...
9994,1,23325,42902992,10,0.000006,17.323080,3.153344e-05,2.913881,1.233099,0.001139,0.016835
9995,1,23325,42903278,10,0.000006,33.845270,5.967399e-09,1.932776,1.643628,0.001139,0.025381
9996,1,23325,43014145,10,0.000006,15.298110,9.180840e-05,3.115007,1.166354,0.001139,0.015748
9997,1,23325,43020938,10,0.000006,46.095880,1.126050e-11,1.555051,1.861077,0.001139,0.031546


In [13]:
with open('concept_xrefs.json') as f:
  xref_data = json.load(f)

In [15]:
xref_data[0]

{'_id': '197339',
 'concept_name': 'Congenital abnormality of uterus, affecting pregnancy',
 'domain_id': 'Condition',
 'concept_class_id': 'Clinical Finding',
 'xrefs': {'SNOMEDCT': '41215002',
  'ICD9CM': ['654.0', '654.00'],
  'UMLS': ['C0269727', 'C0157006']}}

In [19]:
xref_data_dict = {}
for x in xref_data:
    xref_data_dict[x["_id"]] = x

## API 1

In [29]:

paired_concept_file_path = 'pc_test.csv'
paired_concept_column_names = ["dataset_id","concept_id_1","concept_id_2","concept_count","concept_prevalence","chi_square_t","chi_square_p","expected_count","ln_ratio","rel_freq_1","rel_freq_2"]

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def generate_results(concept_dict,xref_data_dict):
    assoc_id = int(concept_dict["concept_id_2"])
    current_result = {
            "associated_concept_id": assoc_id,
            "associated_concept_name": xref_data_dict[str(assoc_id)]["concept_name"],
            "associated_domain_id": xref_data_dict[str(assoc_id)]['domain_id'],
            "concept_count": int(concept_dict["concept_count"]),
            "concept_prevalence": concept_dict["concept_prevalence"],
            "dataset_id": concept_dict["dataset_id"],
            "chi_square_t": concept_dict["chi_square_t"],
            "chi_square_p": concept_dict["chi_square_p"],
            "ln_ratio": concept_dict["ln_ratio"],
            "rel_freq_1": concept_dict["rel_freq_1"],
            "rel_freq_2":concept_dict["rel_freq_2"]
    }
    return current_result 

def load_annotations():
    chunk_size = 1000
    max_combos = 2
    paired_concepts_table_total = pd.read_csv(paired_concept_file_path, chunksize=chunk_size)  
    first_chunk = True
    row_counter = 0
    row_total = file_len(paired_concept_file_path)
    extra_entry = False
    for chunk in paired_concepts_table_total:
        paired_concepts_table = chunk
        for i,j in paired_concepts_table.iterrows():
            row_counter = row_counter + 1
            current_id = int(j["concept_id_1"])
            if(first_chunk):
                last_id = int(paired_concepts_table.iloc[0]["concept_id_1"])
                current_results = []
                current_results.append(generate_results(paired_concepts_table.iloc[0], xref_data_dict))
                current_count = 1
                first_chunk = False
            elif((current_count < max_combos) & (last_id == current_id) & (row_counter != (row_total - 1))):
                current_results.append(generate_results(j,xref_data_dict))
                current_count = current_count + 1
            elif((current_count >= max_combos) & (last_id == current_id) & (row_counter != (row_total - 1))):
#                 print("WOOO")
                current_results.append(generate_results(j,xref_data_dict))
                current_results.pop(0)
                current_count = current_count + 1
            elif((last_id != current_id) | (i == (row_total - 1))):
                last_id_results = current_results
                if((last_id == current_id)):
                    last_id_results.append(generate_results(j))
                    if(len(last_id_results) > max_combos):
                        last_id_results.pop(0)
                elif((last_id != current_id) & (i == (row_total - 1))):
                    extra_entry = True
                    extra_dict = {
                        "_id": current_id,
                        "concept_name": xref_data_dict[str(current_id)]["concept_name"],
                        "domain_id": xref_data_dict[str(current_id)]["domain_id"],
                        "xrefs": xref_data_dict[str(current_id)]["xrefs"],
                        "results": [generate_results(j,xref_data_dict)]
                    }
                else:
                    current_results = []
                    current_results.append(generate_results(j,xref_data_dict))
                    current_count = 1
                current_dict = {
                    "_id": last_id,
                    "concept_name": xref_data_dict[str(last_id)]["concept_name"],
                    "domain_id": xref_data_dict[str(last_id)]["domain_id"],
                    "xrefs": xref_data_dict[str(last_id)]["xrefs"],
                    "results": last_id_results
                }
                print(current_dict)
#                 yield(current_dict)
            last_id = current_id
        if(extra_entry):
            print(extra_dict)
#             yield(extra_dict)

load_annotations()

{'_id': 8507, 'concept_name': 'MALE', 'domain_id': 'Gender', 'xrefs': {'Gender': 'M'}, 'results': [{'associated_concept_id': 8516, 'associated_concept_name': 'Black or African American', 'associated_domain_id': 'Race', 'concept_count': 43498, 'concept_prevalence': 0.02429471, 'dataset_id': 1.0, 'chi_square_t': 88.24868000000002, 'chi_square_p': 5.7721619999999996e-21, 'ln_ratio': -0.03323261, 'rel_freq_1': 0.05770014, 'rel_freq_2': 0.4072885}, {'associated_concept_id': 8527, 'associated_concept_name': 'White', 'associated_domain_id': 'Race', 'concept_count': 182994, 'concept_prevalence': 0.1022067, 'dataset_id': 1.0, 'chi_square_t': 595.8188, 'chi_square_p': 1.3590749999999992e-131, 'ln_ratio': 0.038004500000000004, 'rel_freq_1': 0.2427417, 'rel_freq_2': 0.4373609000000001}]}
{'_id': 8515, 'concept_name': 'Asian', 'domain_id': 'Race', 'xrefs': {'Race': '2'}, 'results': [{'associated_concept_id': 938330, 'associated_concept_name': 'sennosides, USP 8.6 MG Oral Tablet', 'associated_domain

{'_id': 22340, 'concept_name': 'Esophageal varices without bleeding', 'domain_id': 'Condition', 'xrefs': {'SNOMEDCT': '14223005', 'ICD10CM': ['I85.00', 'I85.9'], 'ICD9CM': '456.1', 'UMLS': 'C0267092', 'MONDO': '0021644'}, 'results': [{'associated_concept_id': 2617827, 'associated_concept_name': "Eligible clinician attests to documenting in the medical record they obtained, updated, or reviewed the patient's current medications", 'associated_domain_id': 'Observation', 'concept_count': 1047, 'concept_prevalence': 0.0005847754, 'dataset_id': 1.0, 'chi_square_t': 5937.33, 'chi_square_p': 0.0, 'ln_ratio': 1.944944, 'rel_freq_1': 0.6535581, 'rel_freq_2': 0.006257246}, {'associated_concept_id': 4214956, 'associated_concept_name': 'History of clinical finding in subject', 'associated_domain_id': 'Observation', 'concept_count': 1114, 'concept_prevalence': 0.0006221966000000001, 'dataset_id': 1.0, 'chi_square_t': 2607.609, 'chi_square_p': 0.0, 'ln_ratio': 1.284114, 'rel_freq_1': 0.69538080000000

{'_id': 23034, 'concept_name': 'Neonatal hypoglycemia', 'domain_id': 'Condition', 'xrefs': {'SNOMEDCT': ['52767006', '157147000', '439301000000105', '206675002'], 'ICD10CM': 'P70.4', 'ICD9CM': '775.6', 'UMLS': ['C0158986', 'C0477943'], 'HP': '0001998', 'MedDRA': '10020994', 'NCIT': 'C99237'}, 'results': [{'associated_concept_id': 4134565, 'associated_concept_name': 'Hearing examination', 'associated_domain_id': 'Procedure', 'concept_count': 872, 'concept_prevalence': 0.0004870336, 'dataset_id': 1.0, 'chi_square_t': 39314.69, 'chi_square_p': 0.0, 'ln_ratio': 3.836874, 'rel_freq_1': 0.6702537, 'rel_freq_2': 0.03370179}, {'associated_concept_id': 4014295, 'associated_concept_name': 'Single live birth', 'associated_domain_id': 'Condition', 'concept_count': 994, 'concept_prevalence': 0.0005551736, 'dataset_id': 1.0, 'chi_square_t': 21149.68, 'chi_square_p': 0.0, 'ln_ratio': 3.113306, 'rel_freq_1': 0.7640276999999999, 'rel_freq_2': 0.016346}]}
{'_id': 23164, 'concept_name': 'Disorder of ante

## API 2

In [127]:
def load_annotations():
    chunk_size = 1000
    paired_concepts_table_total = pd.read_csv(paired_concept_file_path, sep='\t', header=None, names= paired_concept_column_names, chunksize=chunk_size)  
    for chunk in paired_concepts_table_total:
        paired_concepts_table = chunk
        for i,j in paired_concepts_table.iterrows():
            current_id_1 = str(int(j["concept_id_1"]))
            current_id_2 = str(int(j["concept_id_2"]))
            current_dict = {
              "_id": current_id_1 + "-" + current_id_2,
              "concept1": {
                 "omop": current_id_1,
                 "xrefs": xref_data_dict[current_id_1]["xrefs"]
              },
              "concept2": {
                 "omop": current_id_2,
                 "xrefs": xref_data_dict[current_id_2]["xrefs"]
              },
              "results": [
                {
                    "concept_count": int(j["concept_count"]),
                    "concept_prevalence": j["concept_prevalence"],
                    "dataset_id": j["dataset_id"],
                    "chi_square_t": j["chi_square_t"],
                    "chi_square_p": j["chi_square_p"],
                    "ln_ratio": j["ln_ratio"],
                    "rel_freq_1": j["rel_freq_1"],
                    "rel_freq_2":j["rel_freq_2"]
                }
              ]
            }
#             print(current_dict)
            yield(current_dict)
            
load_annotations()

<generator object load_annotations at 0x7fe412e0cd60>